In [27]:
# Cell 1 — imports
import h5py
from temporaldata import Data, RegularTimeSeries, Interval, LazyInterval, LazyRegularTimeSeries


In [72]:
# Cell 2 — point to your file
h5_path = "/vol/cortex/cd3/pesaranlab/OCD_Mapping_Foundation/processed_data_upd/processed_data/ocd_symptom_provocation_unfiltered/P3_D3_Tsymptom_provocation_SUPENNS001R01.h5"




In [73]:
"""temporary script"""


# Cell 3 — open once, then do every lazy access inside this block
with h5py.File(h5_path, "r") as f:
    # load lazily
    session = Data.from_hdf5(f)   

    # 1) list top-level groups
    print("Top-level keys:", session.keys())

    # 2) inspect the LFP (RegularTimeSeries under 'ieeg')
    lfp = session.ieeg
    print(f"\nIEEG sampling rate: {lfp.sampling_rate} Hz")
    print("First 10 timestamps:", lfp.timestamps[:10])

    # 3) inspect unit metadata (ArrayDict under 'units')
    units = session.units
    print(f"\nUnits sub-keys: {units.keys()}")
    print("First 5 unit IDs:", units.id[:5])
    print("First 5 brain areas:", units.brain_area[:5])

    # 4) inspect the overall session domain (Interval under 'domain')
    dom = session.domain
    print(f"\nSession domain: start={dom.start[0]:.3f}s   end={dom.end[0]:.3f}s")
# --> file is closed here; no further lazy loads will work outside this block



KeyError: "Unable to synchronously open attribute (can't locate attribute: '_unicode_keys')"

In [ ]:
"""2nd attempt at viewing data"""

# Cell 3 — open & inspect the raw IEEG array
with h5py.File(h5_path, "r") as f:
    ieeg = f["ieeg/data"]
    print("IEEG data shape (segments, channels, samples):", ieeg.shape)
    
    # first segment, first channel, first 10 time-points
    ch0 = ieeg[0, 0, :10]
    print("\nFirst 10 samples of channel 0:\n", ch0)
    
    # first segment, first 5 channels, first 10 time-points
    snippet = ieeg[0, :5, :10]
    print("\nSnippet (5 channels × 10 timepoints):\n", snippet)

IEEG data shape (segments, channels, samples): (248,)


In [76]:
# Cell 3 — inspect raw HDF5 structure
with h5py.File(h5_path, "r") as f:
    def print_struct(name, obj):
        depth = name.count("/")
        indent = "  " * depth
        label = name.split("/")[-1] or "/"
        if isinstance(obj, h5py.Group):
            print(f"{indent}{label}/")
        else:
            print(f"{indent}{label}   shape={obj.shape}, dtype={obj.dtype}")
    print(f"\nRaw HDF5 structure for {h5_path!r}:\n")
    f.visititems(print_struct)


Raw HDF5 structure for '/vol/cortex/cd3/pesaranlab/OCD_Mapping_Foundation/processed_data_upd/processed_data/ocd_symptom_provocation_unfiltered/P3_D3_Tsymptom_provocation_SUPENNS001R01.h5':

brainset/
session/
subject/
units/
  id   shape=(248,), dtype=|S4


In [77]:
# Cell 4 — load with temporaldata and list top-level keys
with h5py.File(h5_path, "r") as f:
    session = Data.from_hdf5(f)

print("Top-level keys in Data object:", session.keys())


KeyError: "Unable to synchronously open attribute (can't locate attribute: '_unicode_keys')"

In [45]:
# Cell 5 — drill into each attribute
for key in session.keys():
    attr = getattr(session, key)
    print(f"\n• {key!r} ({type(attr).__name__})")
    # RegularTimeSeries?
    if hasattr(attr, "sampling_rate"):
        print(f"    sampling_rate = {attr.sampling_rate} Hz")
        print(f"    first timestamps = {attr.timestamps[:10]}")
    # IrregularTimeSeries or ArrayDict?
    elif hasattr(attr, "keys"):
        sub = attr.keys()
        print(f"    sub-keys: {sub}")
        # show a snippet of the first field
        example = sub[0]
        val = getattr(attr, example)
        print(f"    example {example}[:5] = {val[:5]}")


• 'brainset' (Data)
    sub-keys: ['brainsets_version', 'derived_version', 'description', 'id', 'origin_version', 'source', 'temporaldata_version']
    example brainsets_version[:5] = 0.1.0

• 'ieeg' (LazyRegularTimeSeries)
    sampling_rate = 500 Hz


RuntimeError: Unable to synchronously get dataspace (identifier is not of specified type)

In [ ]:
# Cell 6 — if there is a domain interval, print it
if hasattr(session, "domain"):
    dom = session.domain
    print(f"\nOverall session domain: start={dom.start}, end={dom.end}")


In [71]:



import pickle
from temporaldata import Interval

# Path to your .pkl file with a plain Interval object
pkl_path = "/vol/cortex/cd3/pesaranlab/OCD_Mapping_Foundation/processed_data/labels/P3_D5_Tsymptom_provocation_SUPENNS001R01part3.pkl"  # ← update this to your file

# Load the Interval
with open(pkl_path, "rb") as f:
    iv = pickle.load(f)

# Print each interval's start, end, and label vector
for idx, (start, end, label) in enumerate(zip(iv.start, iv.end, iv.label)):
    print(f"Interval {idx}:")
    print(f"  start  = {start}")
    print(f"  end    = {end}")
    print(f"  labels = {label}")


Interval 0:
  start  = -22.298333333333176
  end    = 37.701666666666824
  labels = [4. 6. 6. 4. 6. 6. 0.]
Interval 1:
  start  = 266.2466666666669
  end    = 326.2466666666669
  labels = [0. 0. 0. 3. 0. 2. 0.]
